$$
\def\E{\mathbb{E}}
\def\1{\mathbb{1}}
$$
In questo modello multiperiodale un'azienda fallisce quando alla scadenza del proprio debito il valore degli attivi si trova al di sotto di una soglia fissa K. Nel caso in cui si trovi sopra l'azienda può accedere ad un nuovo finziamento, ovviamente ad uno spread maggiore del precedente qualora gli attivi siano scesi (minore se saliti).
L'azienda rinnova i propri finanziamenti ogni $\tau$ anni, in teoria per sempre. Per motivi numerici è opportuno inserire una scadenza $T >> \tau.$
Ad ogni data di rinnovo $t = i \cdot \tau$ (dove i = 0,...,N), i finanziatori dell'azienda determinano lo spread minimo a cui sono dispositi a prestare soldi dalla seguente relazione:
$$
L(t) = \E\big[L(t)\cdot(1+s(t))\1_{\{A(t+\tau) > K\}} + A(t+\tau)\1_{\{A(t+\tau < K\}}
|\mathcal{F}_t\big]
$$
Lo spread si può quindi determinare come:
$$
s(t) = \frac{1}{L \cdot N(d_2)}\big[ L(t)\cdot \big(\1-N(d_2)\big) - A(t)\cdot\big(1-N(d_1)\big)\big]
$$
dove $L(t)$ è calcolabile partendo dal finanziamento originario in $t = 0$ come:
$$
L(t=i\tau) = \prod_{j=0}^{j=i-1}(1+s(j\tau))L(0)
$$
La valutazione dell'equity potrebbe essere quindi effettuata in modalità forward (ad esempio tramite montecarlo) fino alla scadenza fittizia T attualizzando:
* 0, nel caso in cui esista un A(t) < K
* $max(A(T) - L(T), 0)$ se arriva a scadenza

Questo modello ha una cosenguenza, ovvero che il default non dipende dalla storia del passivo (solo il payoff ne dipende). Vederemo in seguito se possibile cambiare qualche assunzione.


In [147]:
A = 100

In [148]:
K = 50

In [149]:
vol = 0.2

In [150]:
tau = 1.0

In [151]:
L = 90.0

In [152]:
T = 50

In [153]:
import numpy as np
import scipy as sp
import scipy.stats
import math

In [154]:
def s(At, Lt):
    d1 = (math.log(At / K) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / K) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

In [155]:
print s(A,L)

0.000180191679296


In [156]:
def pathGen(runs):
    np.random.seed(1)
    nsteps = int((T / tau) + 1)
    vol2 = vol * vol
    epsilon = np.random.normal(0, 1, (runs, nsteps - 1))
    paths = np.ones((nsteps, runs))
    paths[0,:] = A
    for i in range(nsteps)[1:]:
        paths[i,:] = paths[i-1,:] * np.exp(-0.5*vol2 * tau + vol * math.sqrt(tau) * epsilon[:,i-1])
    return paths

In [157]:
def payoff(path):
    np.seterr('raise')
    # Check for default
    if min(path) < K:
        return 0.
    
    # Compute the spread
    try:
        Lt = L
        for i in range(len(path) - 1):
            spread = s(path[i], Lt)
            Lt = Lt * (1 + spread)
    except:
        return 0
    
    return max(path[-1] - Lt,0)

In [158]:
runs = 10000


In [159]:
def E():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff(paths[:,i])
    return payoffs.mean()
        

In [160]:
print E()

43.4665855053


Questo modello non funziona, dovremmo ottenere un numero vicino a A - L iniziali. Ci sono più motivi:
* è difficile capire qual è il payoff a scadenza: ho messo il max tra A - L e 0 poiché l'equity non può diventare negativa, ma così facendo il problema non è ben specificato. Ci sono path che terminano con A vicino a K, $L(T)$ enorme e nessuno che soffre A - L. Anche togliendo il max dalla condizione finale non sembra risolvere
* L può crescere indiscriminatamente senza portare al fallimento, con problemi (numerici e concettuali) enormi.

E' necessario cambiare le specifiche del modello, inserendo una condizione di fallimento differente Stavo pensando di inserire una condizione sullo spread, ma credo che questo porti alla necessità di risolvere il problema backward.
Come primo tentativo per ovviare a questo problema si può ipotizzare come condizione di fallimento quella in cui gli attivi si trovano al di sotto di una percentuale $X$ dei passivi emessi alla data precedente.

In [161]:
X = 0.4

In [162]:
def s_perc(At, Lt):
    d1 = (math.log(At / (Lt * X)) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(At / (Lt * X)) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    Nd1 = sp.stats.norm.cdf(d1)
    Nd2 = sp.stats.norm.cdf(d2)   
    spread = (Lt*(1-Nd2) - At * (1-Nd1)) / (L*Nd2)
    return spread

Il problema, come nel caso precedente, è quello di trovare un payoff terminale adeguato. Probabilmente la cosa migliore è ipotizzare, per quanto assurdo, che all'ultimo periodo valga il modello uniperiodale alla Merton e valutare la condizione terminale a $T - \tau$ con il valore dell'equity dato dal valore della call. Rispetto a come ho tratto il caso precedente devo quindi inserire un ulteriore funzione per la deteriminazione dello spread e la funzione di pricing dell'equity.

In [163]:
def bs(A, L):
    d1 = (math.log(A / L) + (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    d2 = (math.log(A / L) - (0.5 * vol ** 2) * tau) / (vol * math.sqrt(tau))
    call_price = A * sp.stats.norm.cdf(d1) - L * sp.stats.norm.cdf(d2)   
    put_price = L * sp.stats.norm.cdf(-d2) - A * sp.stats.norm.cdf(-d1)
    return put_price, call_price

In [164]:
def b1(A, L, s):
    put, call = bs(A, L * (1+s))
    return L * (1 + s) - put

In [165]:
def s_merton(A, L):
    res = sp.optimize.minimize(
    lambda s: (b1(A, L, s) - L) ** 2,
    [0.0],
    bounds=[(-0.1, 1.0)])
    return res.x[0]


In [166]:
def e_merton(A, L, s):
    put, call = bs(A, L * (1+s))
    return call

In [167]:
def payoff_perc(path):
    Lt = L
    for i in range(len(path) - 1):
        if path[i+1] < X * Lt:
            # Default
            return 0
        if i == (len(path) - 2):
            spread = s_merton(path[i], Lt)
            return e_merton(path[i], Lt, spread)
        else:
            spread = s_perc(path[i], Lt)

        Lt = Lt * (1 + spread)
    
    return max(path[-1] - Lt,0)

In [168]:
def E_perc():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff_perc(paths[:,i])
    return payoffs.mean()

In [169]:
print E_perc()

48.0665248717


Anche questo modello non funziona. Il problema credo dipenda dal fatto che in realtà queste specifiche non sono corrette. Ad esempio nel modello di Merton base il caso in cui il forward degli asset è inferiore al passivo non è ammissibile: non esiste nessun ente disposto a finanziare questo tipo di aziena. Sotto l'esempio numerico di un'aziena che ha debito per 100 e forward degli asset parti a 99.9.

In [170]:
print s_merton(99.9, 100)

1.0


Ovvero lo spread massimo possibile del risolutore numerico pari al 1000%. Quindi, ipotizzare che valga Merton alla scadenza non è opportuno dato che il valore dell'attivo può scendere al di sotto del passivo. Anzi, la possibilità di un rifinanziamento tramite la sola emissione di nuovi bond in questi casi non è mai verosimile: c'è evidentemente da affrontre il modello in modo molto differente.
Mi vengono in mente due possibilità:

1. La condizione di fallimento, anche in modello multiperiodale, è quella in cui gli attivi si trovino al di sotto delle liabilities da ripagare.
2. Nel caso in cui gli attivi siano al di sotto delle liabilities ma sopra lo il livello di fallimento (ad es. $X \cdot L$), viene effettuato un aumento di capitale. Rimane il problema di cosa succede all'ultimo periodo.

Direi di procedere con la 1. Entrambe i casi possono essere affrontati in due modi differenti:
1. con un rifinanziamento periodico tramite passivo caratterizzato da un'unica scadenza (come assunto finora)
2. ipotizzando che vi sia una struttura a termine del passivo che viene di volta in volta rollata. 

Si procede con 1.
In questo modello, ai fini della valutazione dell'equity e dei bond/spread, dovrebbe essere rilevante solo il primo periodo. Adesso lo verifico empiricamente...


In [171]:
def payoff_multi_merton_1(path):
    Lt = L
    for i in range(len(path) - 1):
        spread = s_merton(path[i], Lt)
        Lt = Lt * (1 + spread)
        if path[i+1] < Lt:
            return 0
        if i == (len(path) - 2):
            return max(path[-1] - Lt,0)


In [172]:
def E_multi_merton_1():
    paths = pathGen(runs)
    payoffs = np.ones(runs)
    for i in range(runs):
        payoffs[i] = payoff_multi_merton_1(paths[:,i])
    return payoffs.mean()

In [173]:
T = 20

In [174]:
print E_multi_merton_1()

10.7570490754


Direi che ci siamo (ci si aspettava un valore prossimo a $A(0) - L(0)$). Se il valore dell'equity è determinato univocamente dal primo periodo, a cosa ci serve un modello multiperiodale? Uno degli obiettivi è quello di capire come varia l'equity quando viene incrementato l'attivo mediante emissione di nuovo finanziamento (cambiando la rischiosità dell'attivo). A titolo di esempio prendiamo il caso in cui l'aziena acquisti, per un ammontare C, un bond risk free con scadenza pari a $\tau$ e rendimento nullo. L'equity prima di questa operazione vale evidentemente $A(0) - L(0)$ e su L viene pagato lo spread di Merton. 
Dopo questa operazione l'equity cambia valore:
$$
E_2 = \E[ \max(0, A(T) + C - L(1+s) - C(1+s')) ]
$$
dove s è lo spread prima dell'operazione, s' quello successivo, per determinare il quale si deve risolvere la seguente equazione:
$$
\begin{align}
C &= \E[ \min(C(1+s'),  (A(T) + C) \frac{C}{L+C}) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C}, A(T) + C) ] \\
&= \frac{C}{L+C} \E[ \min(C(1+s')\frac{L+C}{C} - C, A(T)) + C ] \\
&= \frac{C}{L+C} \E[ \min((1+s')(L+C) - C, A(T)) ] + \frac{C^2}{L+C} \\
\end{align}
$$
dove è stato ipotizzato un recovery proporzionale al face value del debito.

In [175]:
def C_price(A, L, C, sprime):
    strike = (L + C) * (1 + sprime) - C
    put, call = bs(A, strike)
    return C / (L + C) * (strike - put) + (C ** 2) / (L + C)

In [176]:
def e2(A, L, s, sprime):
    put, call = bs(A, L * (1 + s) + C * sprime)
    return call

In [177]:
def sprime(A, L, C):
    res = sp.optimize.minimize(
    lambda s: (C_price(A, L, C, s) - C) ** 2,
    [0.0],
    bounds=[(-0.5, 1.0)])
    return res.x[0]


In [178]:
s = s_merton(A, L)

In [179]:
s

0.066005296966200763

In [180]:
C=20

In [185]:
s_prime = sprime(A, L, C)

In [186]:
s_prime

0.11765978817335848

In [187]:
e2(A, L, s, s_prime)

9.0864645711136518

Quindi l'equity diminuisce. Adesso vediamo cosa succede nell'ipotesi in cui il bond acquistato C abbia una maturity pari a 2 $\tau$. A $t = \tau$ deve essere rifinanziato (nel caso in cui non ci sia fallimento), un importo pari a $L(1+s) + C(1+s_{prime})$ ad un nuovo spread $s_2$. Il nuovo spread soddisfa la seguente equazione:
$$
\begin{align}
L(1+s) + C(1+s_{prime}) = L_{tot}  &= \E\big[\min(L_{tot} \cdot (1 + s_2), A(2\tau) + C)\big]  \\
 & = \E\big[\min(L_{tot} \cdot (1 + s_2) - C, A(2\tau))\big] + C
\end{align}
$$

In [190]:
def C2_price(A, Ltot, C, s2):
    strike = Ltot * (1 + s2) - C
    put, call = bs(A, strike)
    return (strike - put) + C

def s2(A, Ltot, C):
    res = sp.optimize.minimize(
    lambda s: (C2_price(A, Ltot, C, s) - Ltot) ** 2,
    [0.0],
    bounds=[(-0.5, 1.0)])
    return res.x[0]

T = 2.
runs = 10000
A = 100.0
L = 90.0
C = 20.0
# Prima proviamo con un singolo finanziamento lungo tau = 2
print "Bond con maturity = tau"
tau = 1.
s = s_merton(A, L)
s_prime = sprime(A, L, C)
print s, s_prime, e2(A, L, s, s_prime)

print "Bond con maturity = 2*tau"
print "Single financing"
tau = 2.
s = s_merton(A, L)
s_prime = sprime(A, L, C)
print s, s_prime, e2(A, L, s, s_prime)
# Ora con 2 finanziamenti
print "Rolling financing"
tau = 1.0
paths = pathGen(runs)
payoffs = np.zeros(runs)
s = s_merton(A, L)
s_prime = sprime(A, L, C)
totL1 = L*(1+s) + C*(1+s_prime)

for i in range(runs):
    if (paths[1, i] + C) < totL1:
         payoffs[i] = 0.
    else:
        # Find the new spread
        s_2 = s2(paths[1, i], totL1, C)
        payoffs[i] = max(paths[2, i] + C - totL1 * (1 + s_2), 0.)
print s, s_prime, payoffs.mean()


Bond con maturity = tau
0.0660052969662 0.0540043347963 9.42588258816
Bond con maturity = 2*tau
Single financing
0.143806407453 0.117659788173 9.08646457111
Rolling financing
0.0660052969662 0.0540043347963 9.597729068


La diminuzione in valore dell'equity rollando il finanziamento è la medesima (a meno dell'errore montecarlo) che si ha nel caso in cui l'obbligazione C ha una maturity pari a $\tau$: dipende quindi non dalla scadenza del titolo, ma dalla frequenza di rifinanziamento. Questo si poteva evincere anche da quanto osservato prima relativamente al modello rolling di Merton: nel caso in esempio in cui la maturity è pari a $2\tau$, alla data di rifinanziamento $\tau$ il valore dell'equity sarà pari al $\max(A(\tau) + C - L\cdot(1 + s)(1+s'),0)$, esattamente lo stesso del caso in cui il bond dura $\tau$.

Domanda di intermezzo: dopo ogni periodo, se l'azienda ha prodotto utili (i.e. gli asset sono saliti), l'equity potrebbe essere remunerato liquidando parte degli attivi. Questo influisce o meno?